context:
- All images from sakugabooru has been downloaded, with schema:
    -  `./data/post_{id}/post_{id}.{ext}`
    - `./data/post_{id}/post_{id}.json`
- Now we need to upload them to huggingface by tarring it.

To install the libraries:

```bash
pip install unibox hfutils
```

In [1]:
import os
import tarfile
from pathlib import Path
from typing import List
import unibox as ub

get available files:

In [ ]:
# Example usage:
data_dir = "/rmt/yada/dev/sakuga-scraper/data"
output_dir = "/rmt/yada/dev/sakuga-scraper/tars"

# Collect all valid media files
all_files = ub.traverses("/rmt/yada/dev/sakuga-scraper/data")



# Convert extensions to sets for validation
image_extensions = set(ub.IMG_FILES) - {'.gif'}
video_extensions = {'.webm', '.mp4', '.gif'}

# Categorize files
image_files = [f for f in all_files if Path(f).suffix in image_extensions]
video_files = [f for f in all_files if Path(f).suffix in video_extensions]
json_files = [f for f in all_files if Path(f).suffix == '.json']

# Recompute lengths
total_unique_files = len(set(image_files + video_files + json_files))
print(f"all files: {len(all_files)}\nimage files: {len(image_files)}, video files: {len(video_files)}, json files: {len(json_files)}\ntotal unique files: {total_unique_files}")

all files: 404160
image files: 8680, video files: 155238, json files: 240242
total unique files: 404160


In [4]:
json_files[:1], image_files[:1], video_files[:1]

(['/rmt/yada/dev/sakuga-scraper/data/post_100112/sakuga_100112.json'],
 ['/rmt/yada/dev/sakuga-scraper/data/post_122629/sakuga_122629.jpg'],
 ['/rmt/yada/dev/sakuga-scraper/data/post_102725/sakuga_102725.mp4'])

In [6]:
def determine_tar_info(files: List[str], modulo: int = 1000):
    """
    Determine how many full tars can be created and the current last post number.

    Args:
        files (List[str]): List of file paths (images/videos).
        modulo (int): Constant range of IDs per tar file (default: 10,000).

    Returns:
        tuple: (number_of_full_tars, last_post_id)
    """
    post_ids = []
    for file_path in files:
        filename = os.path.basename(file_path)
        try:
            post_id = int(filename.split('_')[1].split('.')[0])
            post_ids.append(post_id)
        except (IndexError, ValueError):
            print(f"Skipping file with invalid format: {filename}")
            continue

    if not post_ids:
        return 0, None

    last_post_id = max(post_ids)
    number_of_full_tars = (last_post_id + 1) // modulo

    return number_of_full_tars, last_post_id


# full tar count, last post id
determine_tar_info(image_files), determine_tar_info(video_files)

((272, 272758), (273, 273264))

do the tarring:

In [7]:
import os
import tarfile
from pathlib import Path
from typing import List

def generate_tar_from_files(
    files: List[str], 
    output_dir: str, 
    id: int, 
    modulo: int = 1000
) -> List[str]:
    """
    Create tar files containing files (images/videos) grouped by post ID ranges.

    Args:
        files (List[str]): List of file paths (images/videos).
        output_dir (str): Directory to store the tar files.
        id (int): The ID used to determine the range of files to include in the tar.
        modulo (int): Constant range of IDs per tar file (default: 10,000).

    Returns:
        List[str]: List of created tar file paths.
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Determine ID range
    range_start = id * modulo
    range_end = (id + 1) * modulo

    # Filter files by ID range
    files_in_range = []
    for file_path in files:
        filename = os.path.basename(file_path)
        try:
            post_id = int(filename.split('_')[1].split('.')[0])
        except (IndexError, ValueError):
            print(f"Skipping file with invalid format: {filename}")
            continue

        if range_start <= post_id < range_end:
            # add the file itself
            files_in_range.append(file_path)

            # add metadata json
            metadata_file = Path(file_path).with_suffix('.json')
            if metadata_file.exists():
                files_in_range.append(str(metadata_file))


    if not files_in_range:
        print(f"No files found in range {range_start}-{range_end}.")
        return []

    tar_file_path = output_dir / f"{id}.tar"

    # Create the tar
    with tarfile.open(tar_file_path, "w") as tar:
        for file_path in files_in_range:
            tar.add(file_path, arcname=os.path.basename(file_path))

    print(f"Created tar: {tar_file_path} with {len(files_in_range)} files.")

    return [str(tar_file_path)]

# Example usage:
# Suppose we have a collection of images & videos:
# files = [
#     "/path/to/sankaku_16465.jpg",
#     "/path/to/sankaku_20000.mp4",
#     ...
# ]

# See how many tars and the highest post ID:
media_files = video_files + image_files
tar_count, last_id = determine_tar_info(media_files)
print("We can make", tar_count, "full tars, last post id is", last_id)

We can make 273 full tars, last post id is 273264


In [15]:
# Create the tar for the first batch (id=0):
!mkdir -p /lv0/tar
tars_created = generate_tar_from_files(media_files, output_dir="/lv0/tars", id=0)
print(tars_created)

Created tar: /lv0/tars/0.tar with 600 files.
['/lv0/tars/0.tar']


make indexes:

In [17]:
from hfutils.index.make import tar_create_index

# Create index for the generated tars
# tar_create_index(src_tar_file="/rmt/yada/dev/sakuga-scraper/tars/0.tar")
tar_create_index(src_tar_file="/lv0/tars/0.tar")

Indexing tar file '/lv0/tars/0.tar' ...: 0it [00:00, ?it/s]

'/lv0/tars/0.json'

testing partial tar read:

In [18]:
import hashlib
import json
import os

def test_local_partial_read(tar_path: str, index_json_path: str):
    """
    Perform local partial reads of an uncompressed tar file to verify offsets and sizes
    match the metadata in the JSON index.

    :param tar_path: Path to the tar file.
    :param index_json_path: Path to the JSON index (the .tar.json file).
    """
    with open(index_json_path, 'r', encoding='utf-8') as jf:
        index_data = json.load(jf)

    # Sanity check: Compare tar size
    reported_size = index_data.get("filesize", None)
    actual_size = os.path.getsize(tar_path)
    if reported_size and reported_size != actual_size:
        raise ValueError(f"Tar size mismatch! JSON says {reported_size} bytes, actual is {actual_size} bytes.")

    with open(tar_path, "rb") as tar_f:
        for fname, info in index_data["files"].items():
            offset = info["offset"]
            size = info["size"]
            expected_sha256 = info["sha256"]

            # Seek to offset and read exactly `size` bytes
            tar_f.seek(offset, 0)
            data = tar_f.read(size)
            if len(data) != size:
                raise ValueError(f"Read {len(data)} bytes instead of {size} for '{fname}'.")

            # Compute SHA-256 and compare
            actual_sha256 = hashlib.sha256(data).hexdigest()
            if actual_sha256 != expected_sha256:
                raise ValueError(f"SHA mismatch for '{fname}'. Expected {expected_sha256}, got {actual_sha256}.")

    print("All files verified successfully via local partial-read checks.")


test_local_partial_read("/lv0/tars/0.tar", "/lv0/tars/0.json")

All files verified successfully via local partial-read checks.


In [8]:
generate_tar_from_files(media_files, output_dir="/lv0/tars", id=273)

Created tar: /lv0/tars/273.tar with 496 files.


['/lv0/tars/273.tar']

tarring the entire set:

In [20]:
from tqdm.auto import tqdm

# since we already created the first tar, we can start from 1
for i in tqdm(range(1, tar_count)):
    tars_created = generate_tar_from_files(media_files, output_dir="/lv0/tars", id=i)
    print(tars_created)
    tar_create_index(src_tar_file=tars_created[0])

    test_local_partial_read(tars_created[0], f"/lv0/tars/{i}.json")

  0%|          | 0/26 [00:00<?, ?it/s]

Created tar: ../tars/1.tar with 3011 files.
['../tars/1.tar']


Indexing tar file '../tars/1.tar' ...: 0it [00:00, ?it/s]

All files verified successfully via local partial-read checks.
Created tar: ../tars/2.tar with 4460 files.
['../tars/2.tar']


Indexing tar file '../tars/2.tar' ...: 0it [00:00, ?it/s]

...

Indexing tar file '../tars/11.tar' ...: 0it [00:00, ?it/s]

All files verified successfully via local partial-read checks.
Created tar: ../tars/12.tar with 6963 files.
['../tars/12.tar']


Indexing tar file '../tars/12.tar' ...: 0it [00:00, ?it/s]

All files verified successfully via local partial-read checks.


KeyboardInterrupt: 

read and put together all jsons:

In [10]:
all_jsons = ub.concurrent_loads(json_files)

Loading batches:   0%|          | 0/240242 [00:00<?, ?it/s]

In [ ]:
len(all_jsons)

In [ ]:
print(hi)